In [1]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
import re
import plotly.plotly as py
import plotly.graph_objs as go
import requests
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import nltk
from nltk import word_tokenize, everygrams
from nltk.util import ngrams
import logging
import csv
import sys
import os
import zipfile
import logging
import time
import datetime

In [2]:
twentyfirst_cent = pd.read_csv('SEC Files/21centfoxinc_sec_files.csv', sep=',', encoding='utf-8')
attinc = pd.read_csv('SEC Files/attinc_sec_files.csv', sep=',', encoding='utf-8')
cbscorp = pd.read_csv('SEC Files/cbscorp_sec_files.csv', sep=',', encoding='utf-8')
comcastcorp = pd.read_csv('SEC Files/comcastcorp_sec_files.csv', sep=',', encoding='utf-8')
verizoncom = pd.read_csv('SEC Files/verizoncom_sec_files.csv', sep=',', encoding='utf-8')

## find the data table from HTML code
Source: https://github.com/ragraw26/Edgar-COMPANY-FILINGS-Web-Scrapping-Data-Analysis/blob/master/Data%20Scrapping/parsingHTML.py

In [11]:
def find_all_tables(page):
    logging.debug('In the function : find_all_tables')
    all_divtables = page.find_all('table')
    find_all_datatables(page, all_divtables)
    return 0


def foldername(page):
    title = page.find('filename').contents[0]
    if ".htm" in title:
        foldername = title.split(".htm")
        logging.debug('In the function : foldername{}'.format(foldername[0]))
        return foldername[0]


def zip_dir(path_dir, path_file_zip=''):
    if not path_file_zip:
        logging.debug('In a function : zip_dir')
        path_file_zip = os.path.join(
            os.path.dirname(path_dir), os.path.basename(path_dir) + '.zip')
    with zipfile.ZipFile(path_file_zip, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(path_dir):
            for file_or_dir in files + dirs:
                zip_file.write(
                    os.path.join(root, file_or_dir),
                    os.path.relpath(os.path.join(root, file_or_dir),
                                    os.path.join(path_dir, os.path.pardir)))


def assure_path_exists(path):
    logging.debug('In a function : assure_path_exists')
    if not os.path.exists(path):
        os.makedirs(path)


def checktag(param):
    setflag = "false"
    datatabletags = ["background", "bgcolor", "background-color",'BGCOLOR','BACKGROUND-COLOR']
    for x in datatabletags:
        if x in param:
            setflag = "true"
    return setflag

def checkheadertag(param):
    logging.debug('In a function : checkheadertag')
    setflag="false"
    datatabletags=["center","bold"]
    for x in datatabletags:
        if x in param:
            setflag="true"
    return setflag


def printtable(table):
    logging.debug('In a function : printtable')
    printtable = []
    printtrs = table.find_all('tr')
    for tr in printtrs:
        data=[]
        pdata=[]
        printtds=tr.find_all('td')
        for elem in printtds:
            x=elem.text;
            x=re.sub(r"['()]","",str(x))
            x=re.sub(r"[$]"," ",str(x))
            if(len(x)>1):
                x=re.sub(r"[—]","",str(x))
                pdata.append(x)
        data=([elem.encode('utf-8') for elem in pdata])
        printtable.append([elem.decode('utf-8').strip() for elem in data])
    return printtable

def find_all_datatables(page, all_divtables):
    logging.debug('In a function : find_all_datatables')
    count = 0
    allheaders=[]
    for table in all_divtables:
        bluetables = []
        trs = table.find_all('tr')
        for tr in trs:
            global flagtr
            if checktag(str(tr.get('style'))) == "true" or checktag(str(tr)) == "true":
                logging.debug('Checking data tables at Row Level')
                bluetables = printtable(tr.find_parent('table'))
                break
            else:
                tds = tr.find_all('td')
                for td in tds:
                    if checktag(str(td.get('style'))) == "true" or checktag(str(td)) == "true":
                        logging.debug('Checking data tables at Column Level')
                        bluetables = printtable(td.find_parent('table'))
                        break
            if not len(bluetables) == 0:
                break
        if not len(bluetables) == 0:
            logging.debug('Total Number of data tables to be created {}'.format(len(bluetables)))
            count += 1
            ptag=table.find_previous('p');
            while ptag is not None and checkheadertag(ptag.get('style'))=="false" and len(ptag.text)<=1:
                ptag=ptag.find_previous('p')
                if checkheadertag(ptag.get('style'))=="true" and len(ptag.text)>=2:
                    global name
                    name=re.sub(r"[^A-Za-z0-9]+","",ptag.text)
                    if name in allheaders:
                        hrcount+=1
                        hrname=name+"_"+str(hrcount)
                        allheaders.append(hrname)
                    else:
                        hrname=name
                        allheaders.append(hrname)
                        break
            folder_name = foldername(page)
            logging.debug('folder created with folder Name{}'.format(folder_name))
            path = str(os.getcwd()) + "/" + folder_name
            logging.debug('Path for csv creation {}'.format(path))
            assure_path_exists(path)
            if(len(allheaders)==0):
                filename=folder_name+"-"+str(count)
            else:
                filename=allheaders.pop()
            csvname=filename+".csv"
            logging.debug('file creation Name{}'.format(csvname))
            csvpath = path + "/" + csvname
            logging.debug('CSV Path for the file creation {}'.format(csvpath))
            with open(csvpath, 'w', encoding='utf-8-sig', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(bluetables)
            zip_dir(path)




In [4]:
#extract Statements of Income, Statements of Cash Flows and Balance Sheets table from raw txt
def extract_tables (htmlpage):
    statements = ["Consolidated Balance Sheets", "Consolidated Statements of Cash Flows", "Consolidated Statements of Income",\
                  'Consolidated Statements of Income Verizon Communications Inc. and Subsidiaries','Consolidated Balance Sheets \
                  Verizon Communications Inc. and Subsidiaries','Consolidated Statements of Cash Flows Verizon Communications Inc. and Subsidiaries',\
                 'Consolidated Balance Sheet','Consolidated Statement of Income','Consolidated Statement of Cash Flows']
    soup = BeautifulSoup(htmlpage,"html.parser")
    all_divtables = soup.find_all('table')
    bluetables = []
    count = 0
    allheaders=[]

    for table in all_divtables:

        divs = table.find_all('div')
        for div in divs:
            if div.text in statements:
                temp = printtable(div.find_parent('table'))

                # remove tables if number of elements are less than 20
                if len(temp)>20:
                #temp=printtable(div.find_parent('table'))                        
                    bluetables.append(printtable(div.find_parent('table')))
    return bluetables
            


In [6]:
def extract_term (bluetables):
    financial_terms = ['Income Before Income Taxes','Interest expense','Total operating revenues','Operating Income','Depreciation and amortization',\
                  'Total current liabilities','Long-Term Debt','Total current assets','Deferred income taxes','Noncontrolling interest',\
                  'Total stockholders equity','Property, Plant and Equipment – Net','Dividends paid','Cash and Cash Equivalents End of Year']  
    temp=[]
    for i in range(len(bluetables)):
        table_1=bluetables[i]
        table_2=[]
        for table_list in table_1:
        #remove empty value from list
            table_list = [var for var in table_list if var]
            table_2.append(table_list)
        df=pd.DataFrame(table_2)
        #extract rows that match value in financial_terms
        df_terms=df.loc[df.iloc[:,0].isin (financial_terms)]
        temp.append(df_terms)
        #final_frame = pd.concat(concat_frame)
    final_frame = pd.concat(temp,axis=0)
    return final_frame
    

## Extracting AT&T 2014 10-K financial terms

In [7]:

htmlpage_ATT_2014_10_K = request.urlopen('https://www.sec.gov/Archives/edgar/data/732717/0000732717-15-000016.txt')
tables_ATT_2014_10_K = extract_tables(htmlpage_ATT_2014_10_K)
ATT_2014_10_K_financial_terms = extract_term (tables_ATT_2014_10_K)
print (ATT_2014_10_K_financial_terms)

                                        0        1        2        3
7                Total operating revenues  132,447  128,752  127,434
14          Depreciation and amortization   18,273   18,395   18,143
16                       Operating Income   11,746   30,479   12,997
19                       Interest expense    3,613    3,940    3,444
23             Income Before Income Taxes    9,960   27,777   10,439
10                  Deferred income taxes    1,142    1,199      NaN
12                   Total current assets   32,028   23,196      NaN
13    Property, Plant and Equipment – Net  112,898  110,968      NaN
28              Total current liabilities   37,282   34,995      NaN
29                         Long-Term Debt   76,011   69,290      NaN
31                  Deferred income taxes   37,544   36,308      NaN
43                Noncontrolling interest      554      494      NaN
44              Total stockholders equity   86,924   91,482      NaN
7           Depreciation and amort

## Extracting AT&T 2010 10-K financial terms

In [740]:
htmlpage_ATT_2017_10_K = request.urlopen('Https://www.sec.gov/Archives/edgar/data/732717/0000732717-11-000014.txt')
tables_ATT_2017_10_K = extract_tables(htmlpage_ATT_2017_10_K)
ATT_2017_10_K_financial_terms = extract_term (tables_ATT_2017_10_K)
print (ATT_2017_10_K_financial_terms)


                                        0        1        2        3
10               Total operating revenues  124,280  122,513  123,443
15          Depreciation and amortization   19,379   19,515   19,673
20                       Interest expense    2,994    3,368    3,369
10                  Deferred income taxes    1,170    1,247      NaN
12                   Total current assets   19,951   25,187      NaN
13    Property, Plant and Equipment – Net  103,196   99,519      NaN
29              Total current liabilities   33,951   36,951      NaN
30                         Long-Term Debt   58,971   64,720      NaN
32                  Deferred income taxes   22,070   23,579      NaN
43                Noncontrolling interest      303      425      NaN
7           Depreciation and amortization   19,379   19,515   19,673
39                         Dividends paid    9,916    9,670    9,507
44  Cash and Cash Equivalents End of Year    1,437    3,741    1,727


In [9]:
ATT_10_K=attinc[attinc['form'] == '10-K']
ATT_10_K

,fdate,cik,findexdate,form,coname,fsize,doccount,sec_full_path
1,1994-03-18,732717,1994-03-31,10-K,SOUTHWESTERN BELL CORP,246114.0,6.0,https://www.sec.gov/Archives/edgar/data/732717...
39,1999-03-12,732717,1999-03-12,10-K,SBC COMMUNICATIONS INC,477801.0,11.0,https://www.sec.gov/Archives/edgar/data/732717...
51,2000-03-10,732717,2000-03-10,10-K,SBC COMMUNICATIONS INC,592823.0,23.0,https://www.sec.gov/Archives/edgar/data/732717...
64,2001-03-12,732717,2001-03-12,10-K,SBC COMMUNICATIONS INC,811410.0,18.0,https://www.sec.gov/Archives/edgar/data/732717...
70,2002-02-28,732717,2002-02-28,10-K,SBC COMMUNICATIONS INC,1575986.0,13.0,https://www.sec.gov/Archives/edgar/data/732717...
79,2003-03-14,732717,2003-03-14,10-K,SBC COMMUNICATIONS INC,2291720.0,19.0,https://www.sec.gov/Archives/edgar/data/732717...
90,2004-03-11,732717,2004-03-11,10-K,SBC COMMUNICATIONS INC,1129653.0,14.0,https://www.sec.gov/Archives/edgar/data/732717...
121,2005-03-11,732717,2005-03-11,10-K,SBC COMMUNICATIONS INC,3539733.0,18.0,https://www.sec.gov/Archives/edgar/data/732717...
149,2006-03-01,732717,2006-03-01,10-K,AT&T INC.,5466944.0,28.0,https://www.sec.gov/Archives/edgar/data/732717...
178,2007-02-26,732717,2007-02-26,10-K,AT&T INC.,11728587.0,31.0,https://www.sec.gov/Archives/edgar/data/732717...


## Extracting all AT&T 10-K financial terms

In [10]:
for j in range(len(ATT_10_K)):
    att_htmlpage = request.urlopen(ATT_10_K.iloc[j]['sec_full_path'])
    att_tables = extract_tables(att_htmlpage)
    if len(att_tables)==0:
        print (ATT_10_K.iloc[j]['findexdate'],':'"Can't extract tables")
    else:
        att_financial_terms = extract_term (att_tables)
        print (ATT_10_K.iloc[j]['findexdate'])
        print (att_financial_terms)

               



1994-03-31 :Can't extract tables
1999-03-12 :Can't extract tables
2000-03-10 :Can't extract tables
2001-03-12 :Can't extract tables
2002-02-28 :Can't extract tables
2003-03-14 :Can't extract tables
2004-03-11 :Can't extract tables
2005-03-11 :Can't extract tables
2006-03-01 :Can't extract tables
2007-02-26 :Can't extract tables
2008-02-27 :Can't extract tables
2009-02-25 :Can't extract tables
2010-02-25 :Can't extract tables
2011-03-01
                                        0        1        2        3
10               Total operating revenues  124,280  122,513  123,443
15          Depreciation and amortization   19,379   19,515   19,673
20                       Interest expense    2,994    3,368    3,369
10                  Deferred income taxes    1,170    1,247      NaN
12                   Total current assets   19,951   25,187      NaN
13    Property, Plant and Equipment – Net  103,196   99,519      NaN
29              Total current liabilities   33,951   36,951      NaN
30      

In [13]:
verizon_10_K=verizoncom[verizoncom['form']=='10-K']
verizon_10_K

,fdate,cik,findexdate,form,coname,fsize,doccount,sec_full_path
6,1994-03-31,732712,1994-03-31,10-K,BELL ATLANTIC CORP,533905.0,17.0,https://www.sec.gov/Archives/edgar/data/732712...
31,1997-03-25,732712,1997-03-25,10-K,BELL ATLANTIC CORP,713054.0,20.0,https://www.sec.gov/Archives/edgar/data/732712...
43,1998-03-25,732712,1998-03-25,10-K,BELL ATLANTIC CORP,142684.0,12.0,https://www.sec.gov/Archives/edgar/data/732712...
61,1999-03-30,732712,1999-03-30,10-K,BELL ATLANTIC CORP,834386.0,19.0,https://www.sec.gov/Archives/edgar/data/732712...
109,2002-03-20,732712,2002-03-20,10-K,VERIZON COMMUNICATIONS INC,498005.0,9.0,https://www.sec.gov/Archives/edgar/data/732712...
131,2003-03-14,732712,2003-03-14,10-K,VERIZON COMMUNICATIONS INC,1733871.0,6.0,https://www.sec.gov/Archives/edgar/data/732712...
147,2004-03-12,732712,2004-03-12,10-K,VERIZON COMMUNICATIONS INC,2161334.0,10.0,https://www.sec.gov/Archives/edgar/data/732712...
170,2005-03-14,732712,2005-03-14,10-K,VERIZON COMMUNICATIONS INC,2293762.0,18.0,https://www.sec.gov/Archives/edgar/data/732712...
212,2006-03-14,732712,2006-03-14,10-K,VERIZON COMMUNICATIONS INC,2451071.0,18.0,https://www.sec.gov/Archives/edgar/data/732712...
239,2007-03-01,732712,2007-03-01,10-K,VERIZON COMMUNICATIONS INC,1984574.0,10.0,https://www.sec.gov/Archives/edgar/data/732712...


## Extracting all Verizon 10-K financial terms

In [16]:
for j in range(len(verizon_10_K)):
    verizon_htmlpage = request.urlopen(verizon_10_K.iloc[j]['sec_full_path'])
    verizon_tables = extract_tables(att_htmlpage)
    if len(verizon_tables)==0:
        print (verizon_10_K.iloc[j]['findexdate'],':'"Can't extract tables")
    else:
        verizon_financial_terms = extract_term (verizon_tables)
        print (verizon_10_K.iloc[j]['findexdate'])
        print (verizon_financial_terms)

1994-03-31 :Can't extract tables
1997-03-25 :Can't extract tables
1998-03-25 :Can't extract tables
1999-03-30 :Can't extract tables
2002-03-20 :Can't extract tables
2003-03-14 :Can't extract tables
2004-03-12 :Can't extract tables
2005-03-14 :Can't extract tables
2006-03-14 :Can't extract tables
2007-03-01 :Can't extract tables
2008-02-28 :Can't extract tables
2009-02-24 :Can't extract tables
2010-02-26 :Can't extract tables
2011-02-28 :Can't extract tables
2012-02-24 :Can't extract tables
2013-02-26 :Can't extract tables
2014-02-27 :Can't extract tables
2015-02-23 :Can't extract tables
2016-02-23 :Can't extract tables
2017-02-21 :Can't extract tables
2018-02-23 :Can't extract tables
